### Installing packages

In [3]:
# Importing packages
import os
import sys
import argparse
import re
import json
import Bio
from Bio import SeqIO
from Bio import Entrez
from tqdm import tqdm

In [4]:
# Asserting name of file.
# In fact shortened form of bacteria name - first letter of genus and nine (or lesser) letters of specie.
json_file_name = "S_ruber.json"
# Asserting email for Entrez.  Print your own mail in the line below
Entrez.email = "vibrio.choleri.1854@gmail.com"

In [5]:
json_path = f"../../data/jsons/{json_file_name}"
with open(json_path, "r") as json_organism:
    json_organism = json.load(json_organism)

# Creating a variables for futher working with links
organism = json_organism[0]
db_search = "assembly"
db_current = "nucleotide"
complete_ids = list(json_organism[1].values())[0]

In [6]:
if type(complete_ids[len(complete_ids)-1]) == list:
    scaffs = complete_ids[len(complete_ids)-1]
    del complete_ids[len(complete_ids)-1]
    complete_ids += scaffs

In [32]:
# Code from source above, with modifications. It is for choosing GenBank nucleotide data only.
def extract_insdc(links):
    linkset = [ls for ls in links[0]['LinkSetDb'] if
              ls['LinkName'] == 'assembly_nuccore_insdc']
    if 0 != len(linkset):
        uids = [link['Id'] for link in linkset[0]['Link']]
    else:
        uids = 0
    return uids

def download_links(db_search, db_current, complete_id, timer, num_link):
    if timer > 0:
        link_handle = Entrez.elink(dbfrom=db_search, db=db_current, from_uid=complete_id)
        link_record = Entrez.read(link_handle)
        uids = extract_insdc(link_record)
        if uids != 0:
            for uid in uids:
                if uid not in links_checked:  # Checking for duplicates
                    links_checked.append(uid)
                    links.append((uid, n))
                    cumulative = 1
                else:
                    cumulative = 0
            num_link += cumulative
    return num_link
            
def forced_download_links(db_search, db_current, complete_id, timer, level, num_link):   # Наверное, это можно реализовать декоратором
    try:
        num_link = download_links(db_search=db_search, db_current=db_current, complete_id=complete_id, timer=timer, num_link=num_link)    
    except RuntimeError:
        print(f"Problem is with {complete_id}")
        timer -= 1
        level += 1
        print(f"We are on the {level} level now")
        num_link = forced_download_links(db_search, db_current, complete_id, timer, level, num_link)
    return num_link

In [33]:
# Taking ids for fetching. It collected all non-duplicated links in nucleotide database from assembly database.
# We use try-except for excepting problems with network temorary lags, which ruined our code
links = []
links_checked = []
n = 0
for complete_id in tqdm(complete_ids):
    timer = 5
    level = 1
    num_link = forced_download_links(db_search, db_current, complete_id, timer, level, num_link)

  0%|                                                                                                                 | 0/38 [00:00<?, ?it/s]

Start_n: 0


  3%|██▊                                                                                                      | 1/38 [00:01<00:45,  1.22s/it]

Start_n: 0


  5%|█████▌                                                                                                   | 2/38 [00:02<00:44,  1.23s/it]

...

Uid: 1419235469
Uid: 1419235469, n: 5
len_list: 19
Final_n: 6
Start_n: 6


 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 37/38 [00:45<00:01,  1.22s/it]

Uid: 295981848
Uid: 295981848, n: 6
len_list: 20
Uid: 294342143
Uid: 294342143, n: 6
len_list: 21
Uid: 294342092
Uid: 294342092, n: 6
len_list: 22
Uid: 294342053
Uid: 294342053, n: 6
len_list: 23
Uid: 294342039
Uid: 294342039, n: 6
len_list: 24
Final_n: 7
Start_n: 7


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:47<00:00,  1.24s/it]

Uid: 83758251
Uid: 83758251, n: 7
len_list: 25
Uid: 83755449
Uid: 83755449, n: 7
len_list: 26
Final_n: 8


In [9]:
# Collecting data about assemblies
gb_records = []
for link in tqdm(links):
    gb_handle = Entrez.efetch(db=db_current, rettype="gb", retmode="text", id=link[0])
    gb_record = SeqIO.read(gb_handle, 'genbank')
    gb_records.append((gb_record, link[1]))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [02:43<00:00,  6.29s/it]


On the next stage we need to create fasta files for each assembly, to reaanotate them.

In [47]:
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"

In [14]:
dna_type, tuples, source_list = [], [], [] # Creating list for identyfing the number of every assemblie DNA molecules (chromosome and any plasmids)
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"
for rec in tqdm(gb_records):
    try:
        source = rec[1] # Number of assembly
        if "plasmid" in rec[0].description:
            dna_type.append("plasmid")
        else:
            dna_type.append("chromosome")
        source_list.append(source) 
        number = source_list.count(source) # Counting the DNA molecule of assembly
        tuples.append((source, number))
        mask = f"{name}{source}_{number}"
        with open (f"../../{name}/data/for_prokka_fasta/{mask}.fasta", "w") as for_prokka_fasta:
            for_prokka_fasta.write(">")
            for_prokka_fasta.write(mask)
            for_prokka_fasta.write("\n")
            for_prokka_fasta.write(str(rec[0].seq))
            for_prokka_fasta.write("\n")
    except Bio.Seq.UndefinedSequenceError:
        print(rec[0].name)
        continue

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2333/2333 [00:00<00:00, 11770.05it/s]

KE340337
KE340338
KE340339
KE340340
KE340341
KE340342
KE340343
KE340344
KE340345
KE340346
KE340347
KI530745
KI530746
KI530747
KI530748
KI530749
KI530750
KI530751
KI530752
KI530753
KI530754
KI530755
KI530756
KI530757
KI530758
KI530759
KI530760
KI530761


In [11]:
# Saving plasmid_code
jsonpc1 = json.dumps(tuples)
with open(f"../../{name}/data/{name}_plasmid_code1.json", "w") as jspc1:
    jspc1.write(jsonpc1)
jsonpc2 = json.dumps(dna_type)
with open(f"../../{name}/data/{name}_plasmid_code2.json", "w") as jspc2:
    jspc2.write(jsonpc2)

In [ ]:
RuntimeError: Couldn't resolve #exLinkSrv2, the address table is empty.